In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

In [8]:
# Read in the modelling input file
model_output = pd.read_csv('../../app/downloads/freight_dual_results_20250414_110751.xls.csv')
model_output.head(2)
model_output.columns

Index(['project_id', 'project_name', 'po_no', 'account', 'account_description',
       'siteid', 'site', 'supplierid', 'suppliername', 'partnumber',
       'partdescription', 'est_commodity_group', 'new_commodity_description',
       'quantity', 'invoice_id', 'invoice_no', 'uom', 'conversion_code',
       'match_supplier', 'est_estimated_area_cost', 'est_estimated_cwt_cost',
       'est_freight_class_area', 'est_freight_class_lbs', 'est_lbs',
       'est_rate_area', 'est_rate_cwt', 'est_sqyd', 'est_uom'],
      dtype='object')

In [ ]:
model_output['xgs(usd/sqyd)'] = None
model_output['xgs(usd/lbs)'] = None
model_output['market(usd/sqyd)'] = None
model_output['market(usd/lbs)'] = None
model_output.columns 

Index(['project_id', 'project_name', 'po_no', 'account', 'account_description',
       'siteid', 'site', 'supplierid', 'suppliername', 'partnumber',
       'partdescription', 'est_commodity_group', 'new_commodity_description',
       'quantity', 'invoice_id', 'invoice_no', 'uom', 'conversion_code',
       'match_supplier', 'est_estimated_area_cost', 'est_estimated_cwt_cost',
       'est_freight_class_area', 'est_freight_class_lbs', 'est_lbs',
       'est_rate_area', 'est_rate_cwt', 'est_sqyd', 'est_uom', 'xgs(usd/sqyd)',
       'xgs(usd/lbs)'],
      dtype='object')

In [ ]:
def calculate_xgs(row):
    if row['est_commodity_group'] == '1VNL':
        try:
            return {'xgs(usd/lbs)': float(row['est_estimated_cwt_cost']) / row['est_lbs'] if row['est_lbs'] > 0 else None,
                    'xgs(usd/sqyd)': None}
        except (ValueError, TypeError):
            return {'xgs(usd/lbs)': None, 'xgs(usd/sqyd)': None}
    elif row['est_commodity_group'] == '1CBL':
        try:
            return {'xgs(usd/sqyd)': float(row['est_estimated_area_cost']) / row['est_sqyd'] if row['est_sqyd'] > 0 else None,
                    'xgs(usd/lbs)': None}
        except (ValueError, TypeError):
            return {'xgs(usd/sqyd)': None, 'xgs(usd/lbs)': None}
    return {'xgs(usd/sqyd)': None, 'xgs(usd/lbs)': None}

# Apply the function to the dataframe
xgs_values = model_output.apply(calculate_xgs, axis=1)
model_output['xgs(usd/lbs)'] = [x['xgs(usd/lbs)'] for x in xgs_values]
model_output['xgs(usd/sqyd)'] = [x['xgs(usd/sqyd)'] for x in xgs_values]

def calculate_market(row):
    if row['est_commodity_group'] == '1VNL':
        try:
            return {'market(usd/lbs)': float(row['market_rate']) / row['est_lbs'] if row['est_lbs'] > 0 else None,
                    'market(usd/sqyd)': None}
        except (ValueError, TypeError):
            return {'market(usd/lbs)': None, 'market(usd/sqyd)': None}
    elif row['est_commodity_group'] == '1CBL':
        try:
            return {'market(usd/sqyd)': float(row['market_rate']) / row['est_sqyd'] if row['est_sqyd'] > 0 else None,
                    'market(usd/lbs)': None}
        except (ValueError, TypeError):
            return {'market(usd/sqyd)': None, 'market(usd/lbs)': None}
    return {'market(usd/sqyd)': None, 'market(usd/lbs)': None}

# Apply the function to the dataframe
xgs_values = model_output.apply(calculate_market, axis=1)
model_output['market(usd/lbs)'] = [x['market(usd/lbs)'] for x in xgs_values]
model_output['market(usd/sqyd)'] = [x['market(usd/sqyd)'] for x in xgs_values]